In [1]:
import matlab.engine
import numpy as np
import pandas as pd
import os

from scipy import optimize
import sklearn.gaussian_process as skg
from scipy.special import erf, erfc
from sklearn.gaussian_process.kernels import RBF, ConstantKernel
import matplotlib.pyplot as plt
from tqdm import tqdm
from numba import jit

import bokeh.io 
from bokeh.plotting import figure, show
from bokeh.layouts import gridplot
from bokeh.transform import linear_cmap
from bokeh.models import LinearColorMapper
bokeh.io.output_notebook()

Loading BokehJS ...

---
# Setting up MATLAB link for simulations

In [2]:
eng = matlab.engine.start_matlab()
eng.cd('.', nargout=0)
eng.cd('../matlab', nargout=0)

In [3]:
obs_loc = matlab.double([[2],[0.1]])
goal = matlab.double([[4],[0]])
alpha = 0.2

out_SD = eng.single_vs_double_integrator(obs_loc, goal, alpha)
out_U = eng.unicycle(obs_loc,goal,alpha)
out = {**out_SD, **out_U}
for x in out.keys():
    out[x] = np.asarray(out[x])

In [4]:
p1 = figure(match_aspect=True)
# p1.line(
#     out["xU"][:,0],
#     out["xU"][:,1],
#     line_width=3,
#     color="blue",
#     legend_label="Unicycle"
# )
p1.line(
    out["xD"][:,0],
    out["xD"][:,1],
    line_width=3,
    color="blue",
    legend_label="Double Integrator Dynamics"
)
p1.line(
    out["xS"][:,0],
    out["xS"][:,1],
    line_width=3,
    color="green",
    legend_label="Single Integrator Dynamics"
)

p1.circle(
    obs_loc[0],
    obs_loc[1],
    radius=0.02,
    color="black"
)

p1.circle(
    obs_loc[0],
    obs_loc[1],
    radius=0.5,
    fill_alpha=0,
    line_width=3,
    color="black"
)

p2 = figure(
    x_axis_label="Time",
    y_axis_label="CBF"
)
# p2.line(
#     out["tU"][:,0],
#     out["hU"][:,0],
#     line_width=3,
#     color="blue"
# )
p2.line(
    out["tD"][:,0],
    out["hD"][:,0],
    line_width=3,
    color="blue"
)
p2.line(
    out["tS"][:,0],
    out["hS"][:,0],
    line_width=3,
    color="green"
)

p2.line(
    [0,60],
    [0,0],
    line_width=3,
    color="black"
)

p1.legend.location = "bottom_left"

grid = gridplot([[p1,p2]])
show(grid)

In [5]:
np.min(out["hU"])

0.05208148401003211

---
# Initialize GPA and define visualization function

In [6]:
lam = 1e-2
l = 0.5
kernel = ConstantKernel(1.0, constant_value_bounds="fixed") * RBF(l, length_scale_bounds="fixed")
gpa = skg.GaussianProcessRegressor(
    alpha=lam,
    kernel=kernel
)

In [7]:
def norm_error(out):
    
    outS = np.array(out["xS"])
    outD = np.array(out["xD"])
    
    xS = outS[:,0]
    yS = outS[:,1]
    xD = outD[:,0]
    yD = outD[:,1]
    
    err = np.sqrt((xS-xD)**2 + (yS-yD)**2)
    
#     p = figure(match_aspect=False)
#     p.line(np.arange(len(xS)),xS)
#     p.line(np.arange(len(xS)),xD,color="red")
    
#     p.line(xS,yS)
#     p.line(xD,yD,color="red")
#     show(p)
    
    return err

In [8]:
def h_error(out):
    
    hS = np.array(out["hS"])
    hD = np.array(out["hD"])
    
    err = hS-hD
    
    return err

In [9]:
def query_di(x,a=0.5):

    # print(x)
    obs_loc = matlab.double([[x[0]],[x[1]]])
    goal = matlab.double([[4],[0]])
    out_SD = eng.single_vs_double_integrator(obs_loc, goal, float(a))
    
    return 0.1+np.min(h_error(out_SD))
    # return np.min(out_SD["hD"])

In [10]:
out = query_di([1,2])
print(out)

-0.014657559275079807


In [11]:
%%capture
n0 = 100
x0 = np.random.uniform(0,2,(n0,1))
y0 = np.random.uniform(-0.5,0.5,(n0,1))

X = np.hstack((x0,y0))
y = np.array([query_di(x.tolist()) for x in X]).reshape(-1,1)
gpa.fit(X,y)

In [12]:
def viz_gpa(gpa,thresh=False, contour=False, return_contour=False, return_thresh=False):

    x1range = [0,2]
    x2range = [-0.5,0.5]
    dx = 0.01
    alpha = 0.5

    nx1 = int(np.diff(x1range)[0]/dx)
    nx2 = int(np.diff(x2range)[0]/dx)
    
    p = figure(
        x_range=[0,2],
        y_range=[-0.5,0.5],
        match_aspect=True,
        plot_width=600,
        plot_height=600*nx2//nx1,
    )

    X1grid,X2grid = np.meshgrid(np.arange(*x1range,dx),np.arange(*x2range,dx))
    Xs = np.vstack([X1grid.ravel(), X2grid.ravel()]).transpose()
    # Xs = np.hstack((Xs, np.ones((Xs.shape[0],1))*alpha))

    # print(Xs)

    ys = gpa.predict(Xs).flatten()

    ys = ys.reshape(nx2,nx1)

    # print(ys)

    color_mapper = LinearColorMapper(palette='Viridis256', low=0, high=2)

    if thresh:
        p.image(
            image=[(ys>0).astype(int)],
            x=x1range[0],
            y=x2range[0],
            dw=np.diff(x1range)[0],
            dh=np.diff(x2range)[0],
            color_mapper = color_mapper
        )
    else:
        p.image(
            image=[ys],
            x=x1range[0],
            y=x2range[0],
            dw=np.diff(x1range)[0],
            dh=np.diff(x2range)[0],
            palette='Viridis256'
        )
    if contour:
        plt.ioff()
        c = plt.contour(ys,[0])
        contsegs = c.allsegs[0][0]
        p.line(
            x=contsegs[:,0]*dx+x1range[0],
            y=contsegs[:,1]*dx+x2range[0],
            line_width=3,
            color="black"
        )
        # print(contour)
    
    if contour and return_contour:
        return [contsegs[:,0]*dx+x1range[0], contsegs[:,1]*dx+x2range[0]]
    elif return_thresh:
        return (ys>0).astype(int)
    else:
        return p

# Ground Truth (Uniform Random, N=2000)

In [13]:
gpa_gt=skg.GaussianProcessRegressor(
    alpha=lam,
    kernel=kernel
)

n0 = 2000
x0 = np.random.uniform(0,2,(n0,1))
y0 = np.random.uniform(-0.5,0.5,(n0,1))

X = np.hstack((x0,y0))
y = np.array([query_di(x.tolist()) for x in X]).reshape(-1,1)
# print(y)
gpa_gt.fit(X,y)

GaussianProcessRegressor(alpha=0.01, kernel=1**2 * RBF(length_scale=0.5))

In [73]:
#Radial Sampling
gpa_gt=skg.GaussianProcessRegressor(
    alpha=lam,
    kernel=kernel
)

n0 = 900
r0 = np.random.uniform(0.25,0.5,(n0,1))
t0 = np.random.uniform(0,2*np.pi,(n0,1))
x0 = r0*np.cos(t0)+0.5
y0 = r0*np.sin(t0)

n1 = 100
x1 = np.random.uniform(0,2,(n1,1))
y1 = np.random.uniform(-0.5,0.5,(n1,1))

x0 = np.vstack((x0,x1))
y0 = np.vstack((y0,y1))

# x0 = np.random.uniform(0,2,(n0,1))
# y0 = np.random.uniform(-0.5,0.5,(n0,1))

X = np.hstack((x0,y0))
y = np.array([query_di(x.tolist()) for x in X]).reshape(-1,1)
# print(y)
gpa_gt.fit(X,y)

GaussianProcessRegressor(alpha=0.01, kernel=1**2 * RBF(length_scale=0.5))

In [74]:
plt.ioff()
p=viz_gpa(gpa_gt,thresh=False, contour=True)
p.circle(
    gpa_gt.X_train_[:,0],
    gpa_gt.X_train_[:,1],
    color="black"
)

show(p)

In [37]:
c_gt = viz_gpa(gpa_gt,thresh=False, contour=True, return_contour=True)

# Uniform Random, N=100

In [17]:
gpa_ur=skg.GaussianProcessRegressor(
    alpha=lam,
    kernel=kernel
)

n0 = 100
x0 = np.random.uniform(0,2,(n0,1))
y0 = np.random.uniform(-0.5,0.5,(n0,1))

X = np.hstack((x0,y0))
y = np.array([query_di(x.tolist()) for x in X]).reshape(-1,1)
# print(y)
gpa_ur.fit(X,y)

GaussianProcessRegressor(alpha=0.01, kernel=1**2 * RBF(length_scale=0.5))

In [18]:
p=viz_gpa(gpa_ur,thresh=False, contour=True)
p.circle(
    gpa_ur.X_train_[:,0],
    gpa_ur.X_train_[:,1],
    color="black"
)

show(p)

# Grid Search, N=100

In [19]:
gpa_gs=skg.GaussianProcessRegressor(
    alpha=lam,
    kernel=kernel
)

n0 = 100
nx = 10
ny = 10

xx = np.arange(0,2,2/nx)+1/nx
yy = np.arange(0,1,1/nx)-0.5+0.5/ny

x0,y0=np.meshgrid(xx,yy)
x0=x0.reshape([-1,1])
y0=y0.reshape([-1,1])


# x0 = np.random.uniform(0,2,(n0,1))
# y0 = np.random.uniform(-0.5,0.5,(n0,1))

X = np.hstack((x0,y0))
y = np.array([query_di(x.tolist()) for x in X]).reshape(-1,1)
# print(y)
gpa_gs.fit(X,y)

GaussianProcessRegressor(alpha=0.01, kernel=1**2 * RBF(length_scale=0.5))

In [20]:
p=viz_gpa(gpa_gs,thresh=False, contour=True)
p.circle(
    gpa_gs.X_train_[:,0],
    gpa_gs.X_train_[:,1],
    color="black"
)

show(p)

In [38]:
x1range = [0,2]
x2range = [-0.5,0.5]
dx = 0.01
alpha = 0.5

nx1 = int(np.diff(x1range)[0]/dx)
nx2 = int(np.diff(x2range)[0]/dx)

X1grid,X2grid = np.meshgrid(np.arange(*x1range,dx),np.arange(*x2range,dx))
Xs = np.vstack([X1grid.ravel(), X2grid.ravel()]).transpose()
# Xs = np.hstack((Xs, np.ones((Xs.shape[0],1))*alpha))

# print(Xs)

ys = gpa.predict(Xs).flatten()

ys = ys.reshape(nx2,nx1)

c = plt.contour(ys,[0])
contour = c.allsegs[0][0]

In [39]:
c_gs = viz_gpa(gpa_gs,thresh=False, contour=True, return_contour=True)

In [40]:
p = figure(
    x_range=[0,2],
    y_range=[-0.5,0.5],
    match_aspect=True,
    plot_width=600,
    plot_height=300
)
p.line(
    x=c_gt[0],
    y=c_gt[1],
    line_width=3,
    color='red'
)
p.line(
    x=c_gs[0],
    y=c_gs[1]
)
show(p)

---
# Set up sampling method

In [26]:
def gpa_next(gpa,x,y):

    n = gpa.X_train_.shape[1]
    g = skg.GaussianProcessRegressor(
        alpha=1e-6,
        kernel=kernel
    )
    g.fit(
        np.vstack((gpa.X_train_, np.array(x).reshape(-1,n))),
        np.vstack((gpa.y_train_, np.array(y).reshape(-1,1)))
    )
    # print(g.X_train_)
    return g

In [27]:
def p_err(mu,sig):
    return 0.5 - erf(np.abs(mu)/(np.sqrt(2)*sig))

In [28]:
# Given a pair (x,y) of "next" data, return the posterior intagrated Perr
def p_err_next(gpa,x,y):
    
    g = gpa_next(gpa,[x],y)
    
    x1range=[0,2] # x position
    x2range=[-0.5,0.5] # y position
    dx = 0.05

    nx1 = int(np.diff(x1range)[0]/dx)
    nx2 = int(np.diff(x2range)[0]/dx)

    X1grid,X2grid = np.meshgrid(np.arange(*x1range,dx),np.arange(*x2range,dx))
    Xs = np.vstack([X1grid.ravel(), X2grid.ravel()]).transpose()

    mus, stds = g.predict(Xs, return_std=True)
    mus = mus[:,0]
    p_err_tp1 = np.sum(p_err(mus,stds))
    
    return p_err_tp1

In [29]:
# Given only a "next" x, assume the sampled y will land exactly on the mean and then return the posterior Perr
@jit
def infer_next(x,args):
    
    gpa = args
    
    n = gpa.X_train_.shape[1]
    yi = gpa.predict(x.reshape(-1,n))
    
    p_err_inf = p_err_next(gpa,x,yi)
    
    return p_err_inf

In [30]:
p_err_next(gpa,np.array([2,0]),0.5)

-3.8935847490528386e+02

In [31]:
bounds = [
    [0,2],
    [-0.5,0.5]
]

opt = optimize.differential_evolution(
    infer_next,
    bounds,
    args=[gpa]
)
opt.x

/tmp/ipykernel_4633/3791675807.py:2: NumbaWarning: 
Compilation is falling back to object mode WITH looplifting enabled because Function "infer_next" failed type inference due to: Untyped global name 'p_err_next': Cannot determine Numba type of <class 'function'>

File "../../../../../tmp/ipykernel_4633/3791675807.py", line 10:
<source missing, REPL/exec in use?>

  @jit
/home/kellan/anaconda3/lib/python3.9/site-packages/numba/core/object_mode_passes.py:151: NumbaWarning: Function "infer_next" was compiled in object mode without forceobj=True.

File "../../../../../tmp/ipykernel_4633/3791675807.py", line 2:
<source missing, REPL/exec in use?>

  warnings.warn(errors.NumbaWarning(warn_msg,
/home/kellan/anaconda3/lib/python3.9/site-packages/numba/core/object_mode_passes.py:161: NumbaDeprecationWarning: 
Fall-back from the nopython compilation path to the object mode compilation path has been detected, this is deprecated behaviour.

For more information visit https://numba.readthedocs.io/

array([0.01220251, 0.00763785])

# Max Error Reduction (N=100)

In [42]:
lam = 1e-2
l = 0.5
kernel = ConstantKernel(1.0, constant_value_bounds="fixed") * RBF(l, length_scale_bounds="fixed")
gpa_di = skg.GaussianProcessRegressor(
    alpha=lam,
    kernel=kernel
)

# Generate random initial data
n0 = 10
x0 = np.random.uniform(0,2,(n0,1))
y0 = np.random.uniform(-0.5,0.5,(n0,1))

X = np.hstack((x0,y0))
y = np.array([query_di(x.tolist()) for x in X]).reshape(-1,1)
gpa_di.fit(X,y)

bounds = [
    [0,2],
    [-0.5,0.5]
]

N = 100
regret_di=[]
for i in tqdm(range(N)):
    
    opt = optimize.differential_evolution(
        infer_next,
        bounds,
        args=[gpa_di]
    )
    
    # opt = optimize.basinhopping(
    #     infer_next,
    #     [1,0,0.5],
        
    x_next = opt.x
    # print(x_next)
    # x_next=[np.random.uniform(0,2),np.random.uniform(-0.5,0.5)]
    
    # print(x_next[2])
    y_next = query_di(x_next)
    
    gpa_di = gpa_next(gpa_di,x_next,y_next)
    
    # regret = np.sum(np.abs(viz_gpa(gpa_di,return_thresh=True)-viz_gpa(gpa_gt2,return_thresh=True)))
    # regret_di.append(regret)
    
    # p = viz_gpa(gpa,[0,2],[-0.5,0.5],0.01,0.5,thresh=True)
    # show(p)

100%|█████████████████████████████████████████| 100/100 [00:28<00:00,  3.56it/s]


In [ ]:
# regret_di1=regret_di

In [34]:
N=(2/.01)*(1/.01)

In [35]:
# plt.plot(np.array(regret_di)/N)
# plt.plot(np.array(regret_di1)/N)
# plt.legend(["Uniform Random","Uncertainty Minimization"])
# plt.xlabel("Iterations")
# plt.ylabel("Classification Error")
# plt.savefig("error.png")
# plt.show()

NameError: name 'regret_di1' is not defined

In [43]:
p = viz_gpa(gpa_di,thresh=False, contour=True)
p.circle(
    gpa_di.X_train_[5:,0],
    gpa_di.X_train_[5:,1],
    color="black"
)
show(p)

# Max Error Reduction (N=1000)

In [ ]:
# lam = 1e-2
# l = 0.5
# kernel = ConstantKernel(1.0, constant_value_bounds="fixed") * RBF(l, length_scale_bounds="fixed")
# gpa_gt2 = skg.GaussianProcessRegressor(
#     alpha=lam,
#     kernel=kernel
# )

# # Generate random initial data
# n0 = 10
# x0 = np.random.uniform(0,2,(n0,1))
# y0 = np.random.uniform(-0.5,0.5,(n0,1))

# X = np.hstack((x0,y0))
# y = np.array([query_di(x.tolist()) for x in X]).reshape(-1,1)
# gpa_gt2.fit(X,y)

# bounds = [
#     [0,2],
#     [-0.5,0.5]
# ]

# N = 500
# for i in tqdm(range(N)):
    
#     opt = optimize.differential_evolution(
#         infer_next,
#         bounds,
#         args=[gpa_gt2]
#     )
    
#     # opt = optimize.basinhopping(
#     #     infer_next,
#     #     [1,0,0.5],
        
#     x_next = opt.x
#     # print(x_next[2])
#     y_next = query_di(x_next)
    
#     gpa_gt2 = gpa_next(gpa_gt2,x_next,y_next)
    
#     # p = viz_gpa(gpa,[0,2],[-0.5,0.5],0.01,0.5,thresh=True)
#     # show(p)

In [46]:
p = viz_gpa(gpa_gt,thresh=False, contour=True)
show(p)

In [47]:
c_gt = viz_gpa(gpa_gt,thresh=False, contour=True, return_contour=True)
c_gs = viz_gpa(gpa_gs,thresh=False, contour=True, return_contour=True)
c_di = viz_gpa(gpa_di,thresh=False, contour=True, return_contour=True)

In [75]:
p = figure(
    x_range=[0,2],
    y_range=[-0.5,0.5],
    match_aspect=True,
    plot_width=600,
    plot_height=300
)

p.line(
    x=c_gt[0],
    y=c_gt[1],
    line_width=3,
    color='red',
    legend_label="Ground Truth (N=2000)"
)
p.line(
    x=c_gs[0],
    y=c_gs[1],
    line_width=3,
    color='blue',
    legend_label="Grid Search (N=100)"
)
# p.circle(
#     gpa_gs.X_train_[:,0],
#     gpa_gs.X_train_[:,1],
#     radius=0.005,
#     color="blue"
# )

p.line(
    x=c_di[0],
    y=c_di[1],
    line_width=3,
    color='green',
    legend_label="Uncertainty Minimization (N=100)"
)
p.circle(
    gpa_di.X_train_[10:,0],
    gpa_di.X_train_[10:,1],
    radius=0.008,
    color="green"
)
show(p)

In [ ]:
def save_data(gpa, fname):
    
    x1range=[0,2] # x position
    x2range=[-0.5,0.5] # y position
    x3range=[0.1,1] # alpha
    dx=0.05
    
    nx1 = int(np.diff(x1range)[0]/dx)
    nx2 = int(np.diff(x2range)[0]/dx)

    X1grid,X2grid,X3grid = np.meshgrid(np.arange(*x1range,dx),np.arange(*x2range,dx),np.arange(*x3range,dx))
    Xs = np.vstack([X1grid.ravel(), X2grid.ravel(), X3grid.ravel()]).transpose()

    mus,stds = gpa.predict(Xs, return_std=True)
    mus = mus.flatten()

    data = np.hstack((Xs,mus.reshape(-1,1),stds.reshape(-1,1)))
    df = pd.DataFrame(
        data=data,
        columns=["x1","x2","alpha","mean","stdev"]
    )
    
    df.to_csv(fname)

In [ ]:
save_data(gpa_di,"gridplot_di.csv")
save_data(gpa_uni,"gridplot_uni.csv")